# NN Model

In [1]:
import numpy as np
from sklearn.metrics import mean_squared_error as mse 
# local imports
from model_tools import PredictorData, TF_NN_Predictor
from utils import Utils

config = Utils.read_config_for_env(config_path='../config/config.yml')
pred_data = PredictorData(
    config,
    refresh_monthly=False,
    refresh_ts_features=False,
    clean_strategy='olrem_for_all',
    # split_strategy='random',
    # split_strategy='last_months_val',
    split_strategy='months',
    num_lag_mon=3,
    val_ratio=0.2,
    scaler_type='standard')

2024-06-08 21:53:38.744867: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-08 21:53:38.753613: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-08 21:53:38.853336: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-08 21:53:41.589333: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Loading data.. Done.
Fixing data schemas.. Done.
Loading /home/onur/WORK/DS/repos/shop_sales_prediction/data/all_ts.parquet


In [2]:
# split the data and do the scaling:
# stores X_train, y_train, X_val, y_val and feature_names in pred_data object
pred_data.split_scale_X_y()

Scaling X train and X val with StandardScaler() scaler


## NN model with 3 Dense layers

In [3]:
tf_nn_predictor_3D = TF_NN_Predictor(
    input_dim=pred_data.X_train.shape[1],
    output_dim=1,
    model_name='3Dense',
    optimizer="adam",
    metrics=['mse'])

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 32)             │           576 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         1,056 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 32)             │         1,056 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,721 (10.63 KB)

 Trainable params: 2,721 (10.63 KB)

 Non-trainable params: 0 (0.00 B)

In [4]:
#Train the model
tf_nn_predictor_3D.model.fit(
    pred_data.X_train,
    pred_data.y_train,
    epochs=10,
    batch_size=4096,
    validation_data=(pred_data.X_val, pred_data.y_val),
    callbacks=tf_nn_predictor_3D.callbacks)

2024-06-08 21:54:37.275644: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 2261340000 exceeds 10% of free system memory.


Epoch 1/10
8118/8119 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.3277 - mse: 0.3277
Epoch 1: val_loss improved from inf to 0.39268, saving model to 3Dense_weights-001--0.39268.keras
8119/8119 ━━━━━━━━━━━━━━━━━━━━ 113s 13ms/step - loss: 0.3276 - mse: 0.3276 - val_loss: 0.3927 - val_mse: 0.3927
Epoch 2/10
8117/8119 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.3144 - mse: 0.3144
Epoch 2: val_loss improved from 0.39268 to 0.38591, saving model to 3Dense_weights-002--0.38591.keras
8119/8119 ━━━━━━━━━━━━━━━━━━━━ 110s 13ms/step - loss: 0.3144 - mse: 0.3144 - val_loss: 0.3859 - val_mse: 0.3859
Epoch 3/10
8118/8119 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.3167 - mse: 0.3167
Epoch 3: val_loss did not improve from 0.38591
8119/8119 ━━━━━━━━━━━━━━━━━━━━ 106s 8ms/step - loss: 0.3167 - mse: 0.3167 - val_loss: 0.3910 - val_mse: 0.3910
Epoch 4/10
8117/8119 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.3115 - mse: 0.3115
Epoch 4: val_loss improved from 0.38591 to 0.38132, saving model to 3Dense_weights-0

In [5]:
# Make predictions on the training and validation data. 
pred_train = tf_nn_predictor_3D.model.predict_on_batch(pred_data.X_train)
pred_val = tf_nn_predictor_3D.model.predict_on_batch(pred_data.X_val)

# Calculate and print the Root Mean Squared Error (RMSE) for training and validation predictions. 
print("Training RMSE: ", np.sqrt(mse(pred_data.y_train, pred_train)))
print("Validation RMSE: ", np.sqrt(mse(pred_data.y_val, pred_val)))

2024-06-08 22:05:50.038899: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 4522680000 exceeds 10% of free system memory.
2024-06-08 22:05:54.591345: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 2261340000 exceeds 10% of free system memory.
2024-06-08 22:05:55.287121: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 4256640000 exceeds 10% of free system memory.
2024-06-08 22:05:59.140721: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 4256640000 exceeds 10% of free system memory.


Training RMSE:  0.54658586
Validation RMSE:  0.6175143
